<h1 align=center><font size = 3>Segmenting and Clustering Neighborhoods in city of Toronto</font></h1>

### Introduction

Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. WE will scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas  dataframe. WE will convert addresses like postalcode and borough information into their equivalent latitude and longitude values. Also, will use the Foursquare API to explore neighborhoods in Toronto City. Will use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters using the k-means clustering algorithm to complete this task. Finally, will use the Folium library to visualize the neighborhoods in Toronto City and their emerging clusters.

<h2 id="DSCW">Downloading And Scraping The Contents Of A Web Page</h2> 


In [19]:
!pip install bs4
#!pip install requests

Import the required modules and functions


In [20]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd

We Download the contents of the web page:

In [21]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

We use <code>get</code> to download the contents of the webpage in text format and store in a variable called <code>data</code>:


In [22]:
data  = requests.get(url).text

We create a <code>BeautifulSoup</code> object using the <code>BeautifulSoup</code> constructor


In [23]:
soup = BeautifulSoup(data,"html5lib")  # create a soup object using the variable 'data'

In [24]:
#find a html table in the web page
table = soup.find('table') # in html table is represented by the tag <table>

We Create a dataframe from the table data consist of three columns: PostalCode, Borough, and Neighborhood.

In [25]:
table_contents=[]
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Print dataframe

In [26]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [27]:
# Shape of the dataframe
df.shape

(103, 3)